<a href="https://colab.research.google.com/github/anhvu1214/Xltn/blob/main/HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install hmmlearn
import os
import librosa
import math
import pickle
import numpy as np
import hmmlearn.hmm
import hmmlearn.hmm as hmm
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [15]:
class_names = ['A', 'B', 'ban', 'len', 'nhay', 'phai', 'trai', 'xuong', 'sil']
states = [3, 3, 6, 6, 6, 6, 6, 9, 6]

def get_mfcc(file_path):
    y, sr = librosa.load(file_path) 
    hop_length = math.floor(sr*0.010) 
    win_length = math.floor(sr*0.025)

    mfcc = librosa.feature.mfcc(y, sr, n_mfcc=12, n_fft=1024, hop_length=hop_length, win_length=win_length)
 
    mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1,1)) 

    delta1 = librosa.feature.delta(mfcc, order=1)
    delta2 = librosa.feature.delta(mfcc, order=2)

    X = np.concatenate([mfcc, delta1, delta2], axis=0) 

    return X.T 
    
length = 0
for d in class_names:
    length += len(os.listdir("/content/drive/MyDrive/Hoc/Xltn/data/" + d))
print(length)

70


In [16]:
all_data = {}
all_labels = {}
for cname in class_names:
    file_paths = [os.path.join("/content/drive/MyDrive/Hoc/Xltn/data/", cname, i) for i in os.listdir(os.path.join('/content/drive/MyDrive/Hoc/Xltn/data/', cname)) if i.endswith('.wav')]
    data = [get_mfcc(file_path) for file_path in file_paths]
    all_data[cname] = data
    all_labels[cname] = [class_names.index(cname) for i in range(len(file_paths))]
  
X = {'train': {}, 'test': {}}
y = {'train': {}, 'test': {}}
for cname in class_names:
    x_train, x_test, _, y_test = train_test_split(
        all_data[cname], all_labels[cname], 
        test_size = 0.33, 
        random_state=42
    )
    X['train'][cname] = x_train
    X['test'][cname] = x_test
    y['test'][cname] = y_test
    
for cname in class_names:
    print(cname, len(X['train'][cname]), len(X['test'][cname]), len(y['test'][cname]))

A 6 4 4
B 2 2 2
ban 4 2 2
len 5 3 3
nhay 4 2 2
phai 6 4 4
trai 5 3 3
xuong 5 3 3
sil 6 4 4


In [21]:
model = {}
for idx, cname in enumerate(class_names):
    start_prob = np.full(states[idx], 0.0)
    start_prob[0] = 1.0
    trans_matrix = np.full((states[idx], states[idx]), 0.0)
    p = 0.5
    np.fill_diagonal(trans_matrix, p)
    np.fill_diagonal(trans_matrix[0:, 1:], 1 - p)
    trans_matrix[-1, -1] = 1.0
    
    #trans matrix
    print(cname)
    print(trans_matrix) 

    # model[cname] = hmm.GaussianHMM(
    #     n_components=states[idx], 
    #     verbose=True, 
    #     n_iter=300, 
    #     startprob_prior=start_prob, 
    #     transmat_prior=trans_matrix,
    #     params='stmc',
    #     init_params='mc',
    #     random_state=42
    # )
    model[cname] = hmm.GMMHMM(n_components=states[idx], n_mix=6,
                           transmat_prior=trans_matrix, startprob_prior=start_prob,
                           covariance_type='diag', n_iter=300)

    model[cname].fit(X=np.vstack(X['train'][cname]), lengths=[x.shape[0] for x in X['train'][cname]])

A
[[0.5 0.5 0. ]
 [0.  0.5 0.5]
 [0.  0.  1. ]]
B
[[0.5 0.5 0. ]
 [0.  0.5 0.5]
 [0.  0.  1. ]]


/usr/local/lib/python3.7/dist-packages/hmmlearn/hmm.py:1051: RuntimeWarning: divide by zero encountered in true_divide
  self.covars_ = c_n / c_d
/usr/local/lib/python3.7/dist-packages/hmmlearn/hmm.py:1051: RuntimeWarning: invalid value encountered in true_divide
  self.covars_ = c_n / c_d
/usr/local/lib/python3.7/dist-packages/scipy/special/_logsumexp.py:112: RuntimeWarning: overflow encountered in exp
  tmp = np.exp(a - a_max)


ban
[[0.5 0.5 0.  0.  0.  0. ]
 [0.  0.5 0.5 0.  0.  0. ]
 [0.  0.  0.5 0.5 0.  0. ]
 [0.  0.  0.  0.5 0.5 0. ]
 [0.  0.  0.  0.  0.5 0.5]
 [0.  0.  0.  0.  0.  1. ]]


/usr/local/lib/python3.7/dist-packages/hmmlearn/hmm.py:1051: RuntimeWarning: invalid value encountered in true_divide
  self.covars_ = c_n / c_d
/usr/local/lib/python3.7/dist-packages/hmmlearn/hmm.py:887: RuntimeWarning: divide by zero encountered in log
  log_cur_weights = np.log(self.weights_[i_comp])


len
[[0.5 0.5 0.  0.  0.  0. ]
 [0.  0.5 0.5 0.  0.  0. ]
 [0.  0.  0.5 0.5 0.  0. ]
 [0.  0.  0.  0.5 0.5 0. ]
 [0.  0.  0.  0.  0.5 0.5]
 [0.  0.  0.  0.  0.  1. ]]


...

/usr/local/lib/python3.7/dist-packages/hmmlearn/hmm.py:1051: RuntimeWarning: divide by zero encountered in true_divide
  self.covars_ = c_n / c_d
/usr/local/lib/python3.7/dist-packages/hmmlearn/hmm.py:1051: RuntimeWarning: invalid value encountered in true_divide
  self.covars_ = c_n / c_d
/usr/local/lib/python3.7/dist-packages/hmmlearn/hmm.py:887: RuntimeWarning: divide by zero encountered in log
  log_cur_weights = np.log(self.weights_[i_comp])
/usr/local/lib/python3.7/dist-packages/scipy/special/_logsumexp.py:112: RuntimeWarning: overflow encountered in exp
  tmp = np.exp(a - a_max)


xuong
[[0.5 0.5 0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.5 0.5 0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.5 0.5 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.5 0.5 0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.5 0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.5 0.5 0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.5 0.5 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.5 0.5]
 [0.  0.  0.  0.  0.  0.  0.  0.  1. ]]


/usr/local/lib/python3.7/dist-packages/hmmlearn/hmm.py:1051: RuntimeWarning: divide by zero encountered in true_divide
  self.covars_ = c_n / c_d
/usr/local/lib/python3.7/dist-packages/hmmlearn/hmm.py:1051: RuntimeWarning: invalid value encountered in true_divide
  self.covars_ = c_n / c_d
/usr/local/lib/python3.7/dist-packages/hmmlearn/hmm.py:887: RuntimeWarning: divide by zero encountered in log
  log_cur_weights = np.log(self.weights_[i_comp])
/usr/local/lib/python3.7/dist-packages/scipy/special/_logsumexp.py:112: RuntimeWarning: overflow encountered in exp
  tmp = np.exp(a - a_max)


sil
[[0.5 0.5 0.  0.  0.  0. ]
 [0.  0.5 0.5 0.  0.  0. ]
 [0.  0.  0.5 0.5 0.  0. ]
 [0.  0.  0.  0.5 0.5 0. ]
 [0.  0.  0.  0.  0.5 0.5]
 [0.  0.  0.  0.  0.  1. ]]


/usr/local/lib/python3.7/dist-packages/hmmlearn/hmm.py:986: RuntimeWarning: invalid value encountered in true_divide
  self.weights_ = w_n / w_d
/usr/local/lib/python3.7/dist-packages/hmmlearn/hmm.py:998: RuntimeWarning: invalid value encountered in true_divide
  self.means_ = m_n / m_d[:, :, None]
Some rows of transmat_ have zero sum because no transition from the state was ever observed.


In [23]:
for cname in class_names:
    name = f'models_train\model_{cname}.pkl'
    with open(name, 'wb') as file: 
        pickle.dump(model[cname], file)

In [24]:
y_true = []
y_pred = []
for cname in class_names:
    for mfcc, target in zip(X['test'][cname], y['test'][cname]):
        scores = [model[cname].score(mfcc) for cname in class_names]
        pred = np.argmax(scores)
        y_pred.append(pred)
        y_true.append(target)
    print((np.array(y_true) == np.array(y_pred)).sum()/len(y_true))
print(y_true)
print(y_pred)

Degenerate mixture covariance


ValueError: ignored

In [ ]:
report = classification_report(y_true, y_pred, target_names=class_names)
print(report)

In [ ]:
model_train = {}
for key in class_names:
    name = f"/content/models_train\model_{key}.pkl"
    with open(name, 'rb') as file:
        model_train[key] = pickle.load(file)
print(model_train)